In [ ]:
import math, sys
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as pp
from PIL import Image
import scipy.fft as fft
import zlib

In [ ]:
filenames = [
	"/home/slembcke/Development/project-drift/tiles/tile3-3461.png",
	"/home/slembcke/Development/project-drift/tiles/tile3-5524.png",
	"/home/slembcke/Development/project-drift/tiles/tile3-6007.png",
	"/home/slembcke/Development/project-drift/tiles/tile3-8058.png",
	"/home/slembcke/Development/project-drift/tiles/tile4-1157.png",
	"/home/slembcke/Development/project-drift/tiles/tile3-5728.png",
	"/home/slembcke/Development/project-drift/tiles/tile3-3938.png",
	"/home/slembcke/Development/project-drift/tiles/tile7-8148.png",
]

zbits = [
	0b0000_00000, 0b0000_00001, 0b0000_00100, 0b0000_00101, 0b0000_10000, 0b0000_10001, 0b0000_10100, 0b0000_10101,
	0b0010_00000, 0b0010_00001, 0b0010_00100, 0b0010_00101, 0b0010_10000, 0b0010_10001, 0b0010_10100, 0b0010_10101,
	0b1000_00000, 0b1000_00001, 0b1000_00100, 0b1000_00101, 0b1000_10000, 0b1000_10001, 0b1000_10100, 0b1000_10101,
	0b1010_00000, 0b1010_00001, 0b1010_00100, 0b1010_00101, 0b1010_10000, 0b1010_10001, 0b1010_10100, 0b1010_10101,
]
def zfunc(n): return 2*zbits[n >> 5] + zbits[n & 0b11111]

zorder, rorder = np.zeros(32*32, int), np.zeros(32*32, int)
for n in range(32*32):
	z = zfunc(n)
	zorder[z], rorder[n] = n, z

def zigzag(n): return (n << 1) ^ (n >> 31)

In [ ]:
q_arr = np.linspace(1, 0, 32)**5
q_arr = 32*q_arr + 16*(1 - q_arr)
pp.plot(q_arr)

q_arr = np.array([q_arr])
q = np.ceil(np.matmul(q_arr.T, q_arr))
q_arr


In [ ]:
def foo(filename):
	img = Image.open(filename)
	z = fft.dctn(img)
	z = np.round(z/q).astype(int)
	h = np.round(fft.idctn(z*q)).astype(int)
	
	bytes = z.flatten().astype(np.int8)
	bytes = bytes[zorder]
	
	return img, bytes, z, h

for filename in filenames:
	img, bytes, z, h, = foo(filename)
	bytes.tofile(filename.replace(".png", ".bin"))
	comp = zlib.compress(bytes, 9)
	print(1024/len(comp))

	_ = pp.hist((h - img).flatten(), 15, (-7, 7))
	pp.show()

	# pp.imshow(abs(z) > 0)
	pp.imshow(abs(bytes.reshape(32, 32)) > 0)
	pp.show()

	pp.imshow(h)
	pp.show()


In [ ]:
img = Image.open(filenames[0])
z = fft.dctn(img)
bytes = (z).flatten()[zorder].astype(int)

In [ ]:
zz = zigzag(bytes)
pp.plot(np.log(zz))
bytes